In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras.utils import np_utils
from keras import regularizers
from keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np

Lecture des données ......... à partir des ensembles de données Keras et définition des données d'entraînement et de test

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

Exploitation des analyses des données: EDA (Exploratory Data Analysis)

In [ ]:
# Vérification du nombre de lignes (records) et de colonnes (features)
print("train_images: ", train_images.shape)
print("train_labels: ",train_labels.shape)
print("test_images: ",test_images.shape)
print("test_labels: ", test_labels.shape)

In [ ]:
# Vérification du nombre de classes uniques
print("train_labels", np.unique(train_labels))
print("test_labels", np.unique(test_labels))

In [ ]:
# Création d'une liste de toutes les étiquettes de classe
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class_names

In [ ]:
# Visualiser certaines des images de l'ensemble de données d'entraînement

plt.figure(figsize=[10,10])
for i in range (25):    # pour les 25 premières  images
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i], cmap=plt.cm.binary)
  plt.xlabel(class_names[train_labels[i][0]])

plt.show()

## Prétraitement des données ( Data Preprocessing ) 
* La raison de la standardisation/normalisation est de convertir toutes les valeurs de pixel en valeurs comprises entre 0 et 1.
* La raison de la conversion de type en float est que to_categorical (hot encodage) a besoin que les données soient de type float par défaut.
* La raison de l'utilisation de to_categorical est que la fonction de perte que nous utiliserons dans ce code (categorical_crossentropy) lors de la compilation du modèle a besoin que les données soient codées à chaud.


In [ ]:
# Conversion des données de pixels en type flottant
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
 
# Standardisation (255 est le nombre total de pixels qu'une image peut avoir)
train_images = train_images / 255
test_images = test_images / 255 

# Encodage  de la classe cible (étiquettes)
num_classes = 10
train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

## Construire le modèle CNN à l'aide de Keras4
### Mise en place des couches

In [ ]:
# Créer un modèle séquentiel et y ajouter des couches

model = Sequential()

model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))    # num_classes = 10

# Vérification du résumé du modèle (summary)
model.summary()

### Compilation du modèle
* Optimiseur utilisé lors de la rétropropagation pour l'ajustement du poids et du biais - Adam (ajuste le taux d'apprentissage de manière adaptative).
* Fonction de perte utilisée - Entropie croisée catégorique (utilisée lorsque plusieurs catégories/classes sont présentes).
* Métriques utilisées pour l'évaluation - Précision.

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

### Ajustement du modèle
* La taille du lot est utilisée pour l'optimiseur Adam.
* Epoch: Époques - Une époque est un cycle complet (passe avant + passe arrière)

In [ ]:
history = model.fit(train_images, train_labels, batch_size=64, epochs=450,
                    validation_data=(test_images, test_labels))

## Visualiser l'évaluation
* Courbe de perte (Loss Curve) - Comparaison de la perte d'entraînement avec la perte de test sur des époques croissantes.
* Courbe de précision ( Accuracy Curve ) - Comparaison de la précision de l'entraînement avec la précision des tests sur des époques croissantes.

In [ ]:
# Loss curve
plt.figure(figsize=[6,4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'green', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

In [ ]:
# Accuracy curve
plt.figure(figsize=[6,4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)